## P7 - Prep the data 

---

In [137]:
# fmt: off


import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

---

## Kernel Kaggle

In [3]:
# Fonction Cleaning Finale
def cleaning(features, test_features, encoding="ohe"):
    """Fonction cleaning finale"""
    # Extract the ids
    train_ids = features["SK_ID_CURR"]
    test_ids = test_features["SK_ID_CURR"]

    # Extract the labels for training
    labels = features["TARGET"]

    # Remove the ids and target
    features = features.drop(columns=["SK_ID_CURR", "TARGET"])
    test_features = test_features.drop(columns=["SK_ID_CURR"])

    # One Hot Encoding
    if encoding == "ohe":
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)

        # Align the dataframes by the columns
        features, test_features = features.align(test_features, join="inner", axis=1)

        # No categorical indices to record
        cat_indices = "auto"

    # Integer label encoding
    elif encoding == "le":
        # Create a label encoder
        label_encoder = LabelEncoder()

        # List for storing categorical indices
        cat_indices = []

        # Iterate through each column
        for i, col in enumerate(features):
            if features[col].dtype == "object":
                # Map the categorical features to integers
                features[col] = label_encoder.fit_transform(
                    np.array(features[col].astype(str)).reshape((-1,))
                )
                test_features[col] = label_encoder.transform(
                    np.array(test_features[col].astype(str)).reshape((-1,))
                )

                # Record the categorical indices
                cat_indices.append(i)

    # Catch error if label encoding scheme is not valid
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")

    print("Training Data Shape: ", features.shape)
    print("Testing Data Shape: ", test_features.shape)

    # Extract feature names
    feature_names = list(features.columns)

    # Impute the domainnomial features
    imputer = SimpleImputer(strategy="median")

    features = imputer.fit_transform(features)
    test_features = imputer.transform(test_features)

    # Scale the domainnomial features
    scaler = MinMaxScaler(feature_range=(0, 1))

    features = scaler.fit_transform(features)
    test_features = scaler.transform(test_features)

    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)

    return feature_names, labels, features, test_features, train_ids, test_ids


In [4]:
# Training data
app_train = pd.read_csv("../../data/raw/application_train.csv")
print("Training data shape: ", app_train.shape)
app_train.head(2)

# Testing data features
app_test = pd.read_csv("../../data/raw/application_test.csv")
print("Testing data shape: ", app_test.shape)
app_test.head(2)

# copy to add fe
app_train_domain = app_train.copy()
app_test_domain = app_test.copy()

# feature engineering with domain knowledge variables
app_train_domain["CREDIT_INCOME_PERCENT"] = (
    app_train_domain["AMT_CREDIT"] / app_train_domain["AMT_INCOME_TOTAL"]
)
app_train_domain["ANNUITY_INCOME_PERCENT"] = (
    app_train_domain["AMT_ANNUITY"] / app_train_domain["AMT_INCOME_TOTAL"]
)
app_train_domain["CREDIT_TERM"] = (
    app_train_domain["AMT_ANNUITY"] / app_train_domain["AMT_CREDIT"]
)
app_train_domain["DAYS_EMPLOYED_PERCENT"] = (
    app_train_domain["DAYS_EMPLOYED"] / app_train_domain["DAYS_BIRTH"]
)

app_test_domain["CREDIT_INCOME_PERCENT"] = (
    app_test_domain["AMT_CREDIT"] / app_test_domain["AMT_INCOME_TOTAL"]
)
app_test_domain["ANNUITY_INCOME_PERCENT"] = (
    app_test_domain["AMT_ANNUITY"] / app_test_domain["AMT_INCOME_TOTAL"]
)
app_test_domain["CREDIT_TERM"] = (
    app_test_domain["AMT_ANNUITY"] / app_test_domain["AMT_CREDIT"]
)
app_test_domain["DAYS_EMPLOYED_PERCENT"] = (
    app_test_domain["DAYS_EMPLOYED"] / app_test_domain["DAYS_BIRTH"]
)

# Create an anomalous flag column
app_train_domain["DAYS_EMPLOYED_ANOM"] = app_train_domain["DAYS_EMPLOYED"] == 365243
# Replace the anomalous values with nan
app_train_domain["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace=True)

app_test_domain["DAYS_EMPLOYED_ANOM"] = app_test_domain["DAYS_EMPLOYED"] == 365243
app_test_domain["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace=True)


app_train_domain["DAYS_EMPLOYED_ANOM"] = app_train_domain["DAYS_EMPLOYED_ANOM"].astype(
    "int32"
)
app_test_domain["DAYS_EMPLOYED_ANOM"] = app_test_domain["DAYS_EMPLOYED_ANOM"].astype(
    "int32"
)

Training data shape:  (307511, 122)
Testing data shape:  (48744, 121)


In [5]:
liste_features, y_train, X_train, X_test, train_ids, test_ids = cleaning(
    app_train_domain, app_test_domain, encoding="ohe"
)
train_final = pd.DataFrame(X_train, columns=liste_features)
train_final["LABELS"] = y_train
train_final["SK_ID_CURR"] = train_ids

test_final = pd.DataFrame(X_test, columns=liste_features)
test_final["SK_ID_CURR"] = test_ids


Training Data Shape:  (307511, 246)
Testing Data Shape:  (48744, 246)


---

## Models

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

### Models scores

In [7]:
from sklearn.metrics import (
    make_scorer,
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)

scoring = {
    "roc_auc": make_scorer(roc_auc_score),
    "accuracy": make_scorer(accuracy_score),
    "precision": make_scorer(precision_score),
    "recall": make_scorer(recall_score),
    "f1": make_scorer(f1_score),
}


---

### SMOTE

In [8]:
from imblearn.over_sampling import SMOTE


smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Check the new class distribution
print("Original class distribution:", pd.Series(y_train).value_counts())
print("Resampled class distribution:", pd.Series(y_train_resampled).value_counts())

Original class distribution: TARGET
0    282686
1     24825
Name: count, dtype: int64
Resampled class distribution: TARGET
1    282686
0    282686
Name: count, dtype: int64


---

In [126]:
# Models
models = [
    (
        "Logistic Regression",
        {"C": 10, "tol": 0.001, "random_state": 42, "verbose": 1, "n_jobs": -1},
        LogisticRegression(),
        (X_train, y_train),
    ),
    (
        "Random Forest",
        {"n_estimators": 100, "random_state": 42, "verbose": 1, "n_jobs": -4},
        RandomForestClassifier(),
        (X_train, y_train),
    ),
    (
        "Random Forest with SMOTE",
        {"n_estimators": 100, "random_state": 42, "verbose": 1, "n_jobs": -4},
        RandomForestClassifier(),
        (X_train_resampled, y_train_resampled),
    ),
]


In [10]:
def evaluate_models(models, cv=5, scoring=scoring):
    """
    Evaluate a list of models using cross-validation and store the results in a dictionary.

    Parameters:
    - models: List of tuples, each containing:
        - model_name: Name of the model (str)
        - params: Dictionary of parameters for the model
        - model: The model instance
        - train_data: Tuple containing (X_train, y_train)
    - cv: Number of cross-validation folds (int)
    - scoring: Scoring metric for cross-validation (str)

    Returns:
    - cv_results_dict: Dictionary containing cross-validation results for each model
    """
    cv_results_dict = {}

    for model_name, params, model, train_data in models:
        X_train = train_data[0]
        y_train = train_data[1]

        cv_results = cross_validate(
            model.set_params(**params),
            X_train,
            y_train,
            cv=cv,
            scoring=scoring,
            return_estimator=True,
        )

        # Store the results in the dictionary
        cv_results_dict[model_name] = cv_results

    return cv_results_dict


cv_results_dict = evaluate_models(models, scoring=scoring)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-4)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-4)]: Done  36 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-4)]: Done 100 out of 100 | elapsed:   12.2s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=-4)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-4)]: Done  36 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-4)]: Done 100 out of 100 | el

---

# MLflow

In [11]:
import mlflow
from mlflow.models import infer_signature

In [139]:
# Initialize MLflow
mlflow.set_tracking_uri("http://localhost:5001")
mlflow.set_experiment("MLflow try2")
# might need to run 'mlflow server --host 127.0.0.1 --port 5001' in the terminal if issues

# With tqdm
for i, element in enumerate(tqdm(models, desc="Processing models")):
    model_name = element[0]
    params = element[1]
    model = element[2]

    # The signature wants the input (X_train) and the output (y_pred)
    # We fit the model so we can use .predict(X_train)
    model.fit(X_train, y_train)
    sample_output = model.predict(X_train)
    signature = infer_signature(X_train, sample_output, params=params)

    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(params)

        if model_name in cv_results_dict:
            metrics = cv_results_dict[model_name]
            mlflow.log_metrics(  # we take the mean of the metric since we have 5 cv results for each model
                {
                    "roc_auc": metrics.get("test_roc_auc", []).mean(),
                    "accuracy": metrics.get("test_accuracy", []).mean(),
                    "precision": metrics.get("test_precision", []).mean(),
                    "recall": metrics.get("test_recall", []).mean(),
                    "f1": metrics.get("test_f1", []).mean(),
                    "fit_time": metrics.get("fit_time", []).mean(),
                }
            )

        mlflow.sklearn.log_model(
            model,
            "model",
            signature=signature,
            registered_model_name=f"{model_name}_reg",
        )

Processing models:   0%|          | 0/3 [00:00<?, ?it/s]2025/04/08 15:43:16 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Registered model 'Logistic Regression_reg' already exists. Creating a new version of this model...
2025/04/08 15:43:16 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Logistic Regression_reg, version 4
Created version '4' of model 'Logistic Regression_reg'.
Processing models:  33%|███▎      | 1/3 [00:08<00:16,  8.10s/it]

🏃 View run Logistic Regression at: http://localhost:5001/#/experiments/201311528320565050/runs/49529b49bfa94fe0974e988b196848ff
🧪 View experiment at: http://localhost:5001/#/experiments/201311528320565050


2025/04/08 15:45:04 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Registered model 'Random Forest_reg' already exists. Creating a new version of this model...
2025/04/08 15:45:05 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Random Forest_reg, version 2
Created version '2' of model 'Random Forest_reg'.
Processing models:  67%|██████▋   | 2/3 [01:57<01:07, 67.51s/it]

🏃 View run Random Forest at: http://localhost:5001/#/experiments/201311528320565050/runs/ebcabb464b244930a389fca70eabc3ce
🧪 View experiment at: http://localhost:5001/#/experiments/201311528320565050


2025/04/08 15:46:54 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Registered model 'Random Forest with SMOTE_reg' already exists. Creating a new version of this model...
2025/04/08 15:46:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Random Forest with SMOTE_reg, version 2
Created version '2' of model 'Random Forest with SMOTE_reg'.
Processing models: 100%|██████████| 3/3 [03:46<00:00, 75.63s/it]

🏃 View run Random Forest with SMOTE at: http://localhost:5001/#/experiments/201311528320565050/runs/cab74b013024431fa28a195796cd9799
🧪 View experiment at: http://localhost:5001/#/experiments/201311528320565050


In [ ]:
# mlflow.end_run()

---